# Cobranza (Recuperación de Cartera)

Notebook para predecir la probabilidad de recuperación de deudas usando datos sintéticos o CSV externo.

## 1. Importar librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, accuracy_score
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
import os
import psutil

# Verificar que todas las librerías estén importadas correctamente
print("✅ Librerías importadas correctamente:")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"XGBoost: {xgb.__version__}")
print(f"Optuna: {optuna.__version__}")

# Verificar hardware disponible
print(f"\n🖥️ Hardware disponible:")
print(f"CPU Cores: {psutil.cpu_count(logical=False)} físicos, {psutil.cpu_count(logical=True)} lógicos")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"RAM disponible: {psutil.virtual_memory().available / (1024**3):.1f} GB")

# Verificar soporte GPU para XGBoost
try:
    import subprocess
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("🎮 GPU NVIDIA detectada - XGBoost puede usar GPU")
        GPU_AVAILABLE = True
    else:
        print("⚠️ GPU NVIDIA no detectada - usando CPU")
        GPU_AVAILABLE = False
except:
    print("⚠️ No se pudo verificar GPU - usando CPU")
    GPU_AVAILABLE = False

print("🚀 Listo para optimización bayesiana optimizada!")

## 2. Cargar datos
### A) Generar datos sintéticos

In [ ]:
np.random.seed(42)
# Generar dataset más grande aprovechando la RAM (64GB)
n = 10000  # 10x más datos para mejor entrenamiento
print(f"📊 Generando dataset de cobranza con {n:,} registros...")

data = pd.DataFrame({
    'dias_atraso': np.random.randint(0, 180, n),  # Más rango de días
    'monto_deuda': np.random.lognormal(6, 1, n),  # Distribución más realista
    'pagos_previos': np.random.poisson(2, n),     # Distribución de Poisson
    'contactos': np.random.poisson(3, n),         # Más contactos
    'segmento': np.random.choice(['Retail','Pyme','Corporativo','Empresarial'], n, p=[0.5, 0.3, 0.15, 0.05]),
    'edad_cliente': np.random.randint(18, 80, n),
    'ingresos_mensuales': np.random.lognormal(8, 0.5, n),
    'score_credito': np.random.normal(650, 100, n),
    'productos_activos': np.random.randint(1, 6, n),
})

# Generar variable objetivo (pago) más realista
pago_prob = (
    0.3 * (data['dias_atraso'] < 30) +                    # Pago temprano
    0.2 * (data['pagos_previos'] > 2) +                   # Historial de pagos
    0.15 * (data['contactos'] > 3) +                      # Muchos contactos
    0.1 * (data['monto_deuda'] < data['ingresos_mensuales'] * 0.3) +  # Deuda baja vs ingresos
    0.1 * (data['score_credito'] > 700) +                 # Buen score crediticio
    0.05 * (data['segmento'].isin(['Corporativo', 'Empresarial'])) +  # Segmento premium
    np.random.rand(n) * 0.1                               # Ruido aleatorio
)

data['pago'] = (pago_prob > 0.4).astype(int)
print(f"✅ Dataset generado: {data.shape[0]:,} filas x {data.shape[1]} columnas")
print(f"📈 Tasa de pago: {data['pago'].mean():.2%}")
data.head()

### B) Cargar datos desde CSV
> Descomenta y ajusta la ruta si tienes tu propio archivo.

In [ ]:
# data = pd.read_csv('deudas_cobranza.csv')
# data.head()

## 3. Análisis exploratorio

In [ ]:
print(data['pago'].value_counts())
data.describe()
data.groupby('segmento')['pago'].mean().plot(kind='bar')
plt.title('Tasa de pago por segmento')
plt.ylabel('Pago Rate')
plt.show()

## 4. Preprocesamiento

In [ ]:
X = data.drop('pago', axis=1)
y = data['pago']
X['segmento'] = LabelEncoder().fit_transform(X['segmento'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

## 5. Modelos
### A) Regresión Logística

In [ ]:
lr = LogisticRegression(max_iter=200)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))

### B) XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

## 6. Evaluación y visualización

## 6. Optimización Bayesiana con Optuna

Vamos a optimizar los hiperparámetros de ambos modelos usando optimización bayesiana para mejorar el rendimiento.


In [ ]:
# Función objetivo para Regresión Logística
def objective_lr(trial):
    # Definir el espacio de búsqueda de hiperparámetros
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet'])
    max_iter = trial.suggest_int('max_iter', 100, 1000)
    
    # Seleccionar solver compatible con la penalty
    if penalty == 'elasticnet':
        solver = 'saga'
    elif penalty == 'l1':
        solver = trial.suggest_categorical('solver_l1', ['liblinear', 'saga'])
    else:  # penalty == 'l2'
        solver = trial.suggest_categorical('solver_l2', ['liblinear', 'lbfgs', 'saga'])
    
    # Crear y entrenar el modelo
    model = LogisticRegression(
        C=C, 
        penalty=penalty, 
        solver=solver, 
        max_iter=max_iter,
        random_state=42
    )
    
    # Validación cruzada con manejo de errores
    try:
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
        return scores.mean()
    except Exception as e:
        return 0.0

# Función para crear modelo optimizado con parámetros mapeados correctamente
def create_optimized_lr_model(best_params):
    """Crea un modelo de regresión logística con los parámetros optimizados"""
    params = {
        'C': best_params['C'],
        'penalty': best_params['penalty'],
        'max_iter': best_params['max_iter'],
        'random_state': 42
    }
    
    if best_params['penalty'] == 'elasticnet':
        params['solver'] = 'saga'
    elif best_params['penalty'] == 'l1':
        params['solver'] = best_params.get('solver_l1', 'liblinear')
    else:  # penalty == 'l2'
        params['solver'] = best_params.get('solver_l2', 'liblinear')
    
    return LogisticRegression(**params)


In [ ]:
# Función objetivo para XGBoost (optimizada para hardware potente)
def objective_xgb(trial):
    # Definir el espacio de búsqueda de hiperparámetros (ampliado para mejor hardware)
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),  # Más árboles
        'max_depth': trial.suggest_int('max_depth', 3, 15),            # Mayor profundidad
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 20),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42,
        'eval_metric': 'logloss'
    }
    
    # Usar GPU si está disponible
    if GPU_AVAILABLE:
        params.update({
            'tree_method': 'gpu_hist',
            'gpu_id': 0,
            'predictor': 'gpu_predictor'
        })
    else:
        params['tree_method'] = 'hist'  # CPU optimizado
    
    # Crear y entrenar el modelo
    model = xgb.XGBClassifier(**params)
    
    # Validación cruzada con manejo de errores
    try:
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
        return scores.mean()
    except Exception as e:
        return 0.0

# Función para crear modelo XGBoost optimizado
def create_optimized_xgb_model(best_params):
    """Crea un modelo XGBoost con los parámetros optimizados"""
    params = best_params.copy()
    params['random_state'] = 42
    params['eval_metric'] = 'logloss'
    return xgb.XGBClassifier(**params)


In [ ]:
# Optimización para Regresión Logística (optimizada para hardware potente)
print("🔍 Optimizando Regresión Logística...")
study_lr = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_lr.optimize(objective_lr, n_trials=200, n_jobs=-1)  # Más trials + paralelización

print(f"Mejor score: {study_lr.best_value:.4f}")
print(f"Mejores parámetros: {study_lr.best_params}")

# Optimización para XGBoost (optimizada para hardware potente)
print("\n🔍 Optimizando XGBoost...")
study_xgb = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_xgb.optimize(objective_xgb, n_trials=200, n_jobs=-1)  # Más trials + paralelización

print(f"Mejor score: {study_xgb.best_value:.4f}")
print(f"Mejores parámetros: {study_xgb.best_params}")


## 7. Modelos Optimizados

Ahora entrenamos los modelos con los mejores hiperparámetros encontrados.


In [ ]:
# Entrenar Regresión Logística optimizada
lr_optimized = create_optimized_lr_model(study_lr.best_params)
lr_optimized.fit(X_train, y_train)
y_pred_lr_opt = lr_optimized.predict(X_test)

print("📊 Regresión Logística Optimizada:")
print(classification_report(y_test, y_pred_lr_opt))
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr_opt):.4f}")

# Entrenar XGBoost optimizado
xgb_optimized = create_optimized_xgb_model(study_xgb.best_params)
xgb_optimized.fit(X_train, y_train)
y_pred_xgb_opt = xgb_optimized.predict(X_test)

print("\n📊 XGBoost Optimizado:")
print(classification_report(y_test, y_pred_xgb_opt))
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb_opt):.4f}")


In [ ]:
# Comparación de accuracy
results_comparison = pd.DataFrame({
    'Modelo': ['Regresión Logística Original', 'Regresión Logística Optimizada', 
               'XGBoost Original', 'XGBoost Optimizado'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_lr_opt),
        accuracy_score(y_test, y_pred_xgb),
        accuracy_score(y_test, y_pred_xgb_opt)
    ]
})

print("📈 Comparación de Resultados:")
print(results_comparison)

# Visualización de la comparación
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.bar(results_comparison['Modelo'], results_comparison['Accuracy'])
plt.title('Comparación de Accuracy - Cobranza')
plt.ylabel('Accuracy')
plt.xticks(rotation=45, ha='right')
plt.ylim(0.5, 0.9)

# Mejora en accuracy
improvements = [
    results_comparison.iloc[1, 1] - results_comparison.iloc[0, 1],  # LR improvement
    results_comparison.iloc[3, 1] - results_comparison.iloc[2, 1]   # XGB improvement
]

plt.subplot(1, 2, 2)
plt.bar(['Regresión Logística', 'XGBoost'], improvements, color=['blue', 'green'])
plt.title('Mejora en Accuracy')
plt.ylabel('Mejora')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

print(f"\n🎯 Mejoras obtenidas:")
print(f"Regresión Logística: +{improvements[0]:.4f} ({improvements[0]*100:.2f}%)")
print(f"XGBoost: +{improvements[1]:.4f} ({improvements[1]*100:.2f}%)")


In [ ]:
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.title('Confusión LR')
plt.imshow(confusion_matrix(y_test, y_pred_lr), cmap='Blues')
plt.subplot(1,2,2)
plt.title('Confusión XGBoost')
plt.imshow(confusion_matrix(y_test, y_pred_xgb), cmap='Greens')
plt.show()
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr.predict_proba(X_test)[:,1])
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_model.predict_proba(X_test)[:,1])
plt.plot(fpr_lr, tpr_lr, label='LogisticReg')
plt.plot(fpr_xgb, tpr_xgb, label='XGBoost')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.title('Curvas ROC')
plt.show()

## 7. Importancia de variables
### XGBoost

In [ ]:
xgb.plot_importance(xgb_model)
plt.show()

## 8. Comentarios finales
- Puedes modificar los parámetros, probar con CSV propio y agregar nuevas visualizaciones.